#install

In [1]:
pip install requests

In [2]:
pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227138 sha256=6d51a384ef22585902503424a7e9cd3153fb13bc6510d5838aa37241c73a686f
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [3]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=9dd3f634b1d533be591adc001457f247428c21072b91f63b56a2d49f082b2279
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [4]:
pip install gensim

#import

In [1]:
import pandas as pd
import numpy as np
import re
import shutil
import os
import nltk
import requests
import kagglehub
import io
import sklearn
import fasttext
import zipfile
import gensim.downloader
from gensim.models import Word2Vec
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from gensim import models
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
from nltk.corpus import stopwords

#download models

##glove

In [6]:
# Download latest version
path = kagglehub.model_download("edithram23/glove/other/wordglove")
print("Path to model files:", path)

100%|██████████| 509M/509M [00:05<00:00, 93.7MB/s]
Extracting model files...


Path to model files: /root/.cache/kagglehub/models/edithram23/glove/other/wordglove/1


In [7]:
# Define paths
downloaded_files_dir = '/root/.cache/kagglehub/models/edithram23/glove/other/wordglove/1/glove.6B.200d.word2vec'
project_dir = '/content'
# Move file from downloaded directory to project directory
shutil.move(downloaded_files_dir, os.path.join(project_dir, 'glove_model.txt'))


'/content/glove_model.txt'

##word2vec

In [8]:
word2vec_model = gensim.downloader.load('word2vec-google-news-300')
word2vec_model.save_word2vec_format("word2vec_model.bin", binary=True)

[==================================================] 100.0% 1662.8/1662.8MB downloaded


##fasttext

In [9]:
zip_file = "crawl-300d-2M-subword.zip"

!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip

zip_path = "/content/" + zip_file

extract_path = "/content/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

os.remove(zip_path)


--2024-05-06 16:51:58--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.162.163.11, 3.162.163.34, 3.162.163.19, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.162.163.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5828358084 (5.4G) [application/zip]
Saving to: ‘crawl-300d-2M-subword.zip’

crawl-300d-2M-subwo 100%[===================>]   5.43G  51.6MB/s    in 1m 58s  

2024-05-06 16:53:57 (47.0 MB/s) - ‘crawl-300d-2M-subword.zip’ saved [5828358084/5828358084]



#Defs

##def_load_word_embedding_model

In [10]:
def load_word_embedding_model(model_type, model_path):
    if model_type == "word2vec":
        return KeyedVectors.load_word2vec_format(model_path, binary=True)
    elif model_type == "word2vec 2":
        return Word2Vec.load(word2vec_2_model_path)
    elif model_type == "glove":
        return KeyedVectors.load_word2vec_format(model_path)
    elif model_type == "fasttext":
        return fasttext.load_model(model_path)
    else:
        raise ValueError("Invalid model type. Supported types: 'word2vec', 'word2vec 2', 'glove', 'fasttext'")

##def_create_feature_vectors

In [11]:
def create_feature_vectors(corpus, word_embedding_model):
    review_vectors = []
    for review in corpus:
        words = review.split()
        review_vector = np.zeros(word_embedding_model.vector_size)
        count = 0
        for word in words:
            print(word_embedding_model)
            if word in word_embedding_model:
                review_vector += word_embedding_model[word]
                count += 1
        if count != 0:
            review_vector /= count
        review_vectors.append(review_vector)
    X = np.vstack(review_vectors)
    return X

In [12]:
def create_feature_vectors_W2v(corpus, word_embedding_model):
    review_vectors = []
    for review in corpus:
        words = review.split()
        review_vector = np.zeros(word_embedding_model.vector_size)
        count = 0
        for word in words:
            if word in word_embedding_model.wv.key_to_index:
                review_vector += word_embedding_model.wv.get_vector(word)
                count += 1
        if count != 0:
            review_vector /= count
        review_vectors.append(review_vector)
    X = np.vstack(review_vectors)
    return X

##clean_reviews

In [20]:
def clean_reviews(no_reviews,language, words_to_be_excluded, words_to_be_included, dataset):

    nltk.download('stopwords') #Must only be downloaded once
     #Includes a list of words that souldn't appear in the reivews

    #Create own list of stopwords
    my_stopwords = stopwords.words(language)
    #Append the words from to_be_excluded_words to my_stopwords
    for word in words_to_be_excluded:
        if not any ( word in stpwords for stpwords in my_stopwords):
            my_stopwords.append(word)

    #Remove words from my_stopwords that are present in to_be_included
    for word in words_to_be_included:
        if any (word in stpwords for stpwords in my_stopwords):
            my_stopwords.remove(word)

    #Stemming is the process of extracting the stem of each word e.g. "loved"->"love"


    corpus =[] #Will contain the clean reviews"
    for i in range (0, no_reviews):
        review = re.sub('[^a-zA-Z]',' ', dataset['Review'][i]) #Removes all other characters besides the letters and puts the review in a string
        review = review.lower() #Transforms all UPPER-CASE characters to lower-case
        review = review.split() #Separates all the words from the review string and puts them in a list
        ps = PorterStemmer()
        review = [ps.stem(word) for word in review if not word in set (my_stopwords)] #Remove irrelevant words from rewiews and stems all words
        review = ' '.join(review)
        corpus.append(review) #Append review string to corpus
    return corpus

##Def_score_acc

In [1]:
def score_acc(cm):
    TN= cm[0][0] #True_Negative or results that were predicted to be negative and were negative
    TP = cm[1][1]#True_Positive or results that were predicted to be positive and were positive

    FN = cm[0][1] #False_Negative or results that were predicted to be negative but were positive
    FP = cm [1][0]#False_Positive or results that were predicted to be positive but were negative
    accuracy = (TP + TN)/(TP + TN + FP + FN) #Or correct_predictions/total_predictions
    precision = TP / (TP + FP) #Or correct_positive_predictions/total_correct_predictions
    recall = TP / (TP + FN)#Or correct_positive_predictions/(correct_positive_predictions + false_negative_predictions)
    score = 2 * precision * recall/(precision + recall)
    return accuracy,score

##Def_naive_bayes

In [22]:
#Applying the Naive Bayes classification algorithm
def naive_bayes(X_train, y_train, X_test, y_test):

    classifier = GaussianNB()
    classifier.fit(X_train, y_train)

    # Predicting the Test set results
    y_pred = classifier.predict(X_test)

    # Making the Confusion Matrix

    cm = confusion_matrix(y_test, y_pred) #Contains results of predictions
    accuracy,score=score_acc(cm)
    print("Accuracy of the Naive Bayes algorithm is ",  accuracy, " and score ", score)
    return accuracy, score


##Def_random_forest

In [23]:
def random_forest(number_of_trees, X_train, y_train, X_test, y_test):

    classifier = RandomForestClassifier(n_estimators = number_of_trees, criterion = 'entropy')
    classifier.fit(X_train, y_train)

    # Predicting the Test set results
    y_pred = classifier.predict(X_test)

    # Making the Confusion Matrix

    cm = confusion_matrix(y_test, y_pred)
    accuracy,score=score_acc(cm)
    print("Accuracy of the Random Forest algorithm for ", number_of_trees, " is ",  accuracy, " and score ", score)
    return accuracy, score

##Def_logistic_regression

In [24]:
def logistic_regression(X_train, y_train, X_test, y_test):

    classifier = LogisticRegression(random_state = 0)
    classifier.fit(X_train, y_train)

    # Predicting the Test set results
    y_pred = classifier.predict(X_test)

    # Making the Confusion Matrix

    cm = confusion_matrix(y_test, y_pred)
    accuracy,score=score_acc(cm)
    print("Accuracy of the Logistic Regression algorithm is ",  accuracy, " and score ", score)
    return accuracy, score

##Def_demo

In [13]:
# Set your parameters
path = '/content/Restaurant_Reviews.tsv'
language = 'english'
words_to_be_included = ['not', 'nor', 'no']
words_to_be_excluded = ['opinion']
no_reviews = 1000
test_set_size = 0.2

In [14]:
word2vec_model_path = '/content/word2vec_model.bin'
word2vec_2_model_path = '/content/W2Vmodel.model'
glove_model_path = '/content/glove_model.txt'
fasttext_model_path = '/content/crawl-300d-2M-subword.bin'

In [15]:
word2vec_model = load_word_embedding_model("word2vec", word2vec_model_path)

In [16]:
word2vec_2_model = load_word_embedding_model("word2vec 2", word2vec_2_model_path)


In [17]:
glove_model = load_word_embedding_model("glove", glove_model_path)


In [ ]:
# fasttext_model = load_word_embedding_model("fasttext", fasttext_model_path)

In [18]:
# Importing the dataset
dataset = pd.read_csv(path, delimiter='\t', quoting=3)

In [25]:
corpus = clean_reviews(no_reviews, language, words_to_be_excluded, words_to_be_included, dataset)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
X_word2vec = create_feature_vectors(corpus, word2vec_model)


In [26]:
X_word2vec_2 = create_feature_vectors_W2v(corpus, word2vec_2_model)

In [ ]:
X_glove = create_feature_vectors(corpus, glove_model)


In [ ]:
# X_fasttext = create_feature_vectors(corpus, fasttext_model)


In [29]:
y = dataset.iloc[:, 1].values


In [30]:
X_train_word2vec, X_test_word2vec, y_train, y_test = train_test_split(X_word2vec, y, test_size=test_set_size)
X_train_word2vec_2, X_test_word2vec_2, _, _ = train_test_split(X_word2vec_2, y, test_size=test_set_size)
X_train_glove, X_test_glove, _, _ = train_test_split(X_glove, y, test_size=test_set_size)
# X_train_fasttext, X_test_fasttext, _, _ = train_test_split(X_fasttext, y, test_size=test_set_size)

In [31]:
print("Naive Bayes - Word2Vec:")
accuracy_bayes_word2vec, score_bayes_word2vec = naive_bayes(X_train_word2vec, y_train, X_test_word2vec, y_test)
print("Naive Bayes - Word2Vec 2:")
accuracy_bayes_word2vec_2, score_bayes_word2vec_2 = naive_bayes(X_train_word2vec_2, y_train, X_test_word2vec_2, y_test)
print("Naive Bayes - GloVe:")
accuracy_bayes_glove, score_bayes_glove = naive_bayes(X_train_glove, y_train, X_test_glove, y_test)
# print("Naive Bayes - FastText:")
# accuracy_bayes_fasttext, score_bayes_fasttext = naive_bayes(X_train_fasttext, y_train, X_test_fasttext, y_test)

Naive Bayes - Word2Vec:
Accuracy of the Naive Bayes algorithm is  0.675  and score  0.6285714285714286
Naive Bayes - Word2Vec 2:
Accuracy of the Naive Bayes algorithm is  0.51  and score  0.4367816091954023
Naive Bayes - GloVe:
Accuracy of the Naive Bayes algorithm is  0.51  and score  0.5148514851485149


In [34]:
print("Random Forest - Word2Vec:")
accuracy_random_forest_word2vec, score_random_forest_word2vec = random_forest(1000, X_train_word2vec, y_train, X_test_word2vec, y_test)
print("Random Forest - Word2Vec 2:")
accuracy_random_forest_word2vec_2, score_random_forest_word2vec_2 = random_forest(1000, X_train_word2vec_2, y_train, X_test_word2vec_2, y_test)  # اضافه شده
print("Random Forest - GloVe:")
accuracy_random_forest_glove, score_random_forest_glove = random_forest(1000, X_train_glove, y_train, X_test_glove, y_test)
# print("Random Forest - FastText:")
# accuracy_random_forest_fasttext, score_random_forest_fasttext = random_forest(1000, X_train_fasttext, y_train, X_test_fasttext, y_test)


Random Forest - Word2Vec:
Accuracy of the Random Forest algorithm for  1000  is  0.7  and score  0.6875
Random Forest - Word2Vec 2:
Accuracy of the Random Forest algorithm for  1000  is  0.545  and score  0.5027322404371584
Random Forest - GloVe:
Accuracy of the Random Forest algorithm for  1000  is  0.47  and score  0.48543689320388345


In [35]:
print("Logistic Regression - Word2Vec:")
accuracy_LR_word2vec, score_LR_word2vec = logistic_regression(X_train_word2vec, y_train, X_test_word2vec, y_test)
print("Logistic Regression - Word2Vec 2:")
accuracy_LR_word2vec_2, score_LR_word2vec_2 = logistic_regression(X_train_word2vec_2, y_train, X_test_word2vec_2, y_test)
print("Logistic Regression - GloVe:")
accuracy_LR_glove, score_LR_glove = logistic_regression(X_train_glove, y_train, X_test_glove, y_test)
# print("Logistic Regression - FastText:")
# accuracy_LR_fasttext, score_LR_fasttext = logistic_regression(X_train_fasttext, y_train, X_test_fasttext, y_test)

Logistic Regression - Word2Vec:
Accuracy of the Logistic Regression algorithm is  0.735  and score  0.7195767195767195
Logistic Regression - Word2Vec 2:
Accuracy of the Logistic Regression algorithm is  0.535  and score  0.5550239234449761
Logistic Regression - GloVe:
Accuracy of the Logistic Regression algorithm is  0.52  and score  0.5151515151515151


In [37]:
print("Naive Bayes - Word2Vec - Accuracy:", accuracy_bayes_word2vec, "F1 Score:", score_bayes_word2vec)
print("Naive Bayes - Word2Vec 2 - Accuracy:", accuracy_bayes_word2vec_2, "F1 Score:", score_bayes_word2vec_2)
print("Naive Bayes - GloVe - Accuracy:", accuracy_bayes_glove, "F1 Score:", score_bayes_glove)
# print("Naive Bayes - FastText - Accuracy:", accuracy_bayes_fasttext, "F1 Score:", score_bayes_fasttext)

Naive Bayes - Word2Vec - Accuracy: 0.675 F1 Score: 0.6285714285714286
Naive Bayes - Word2Vec 2 - Accuracy: 0.51 F1 Score: 0.4367816091954023
Naive Bayes - GloVe - Accuracy: 0.51 F1 Score: 0.5148514851485149


In [38]:
print("Random Forest - Word2Vec - Accuracy:", accuracy_random_forest_word2vec, "F1 Score:", score_random_forest_word2vec)
print("Random Forest - Word2Vec 2 - Accuracy:", accuracy_random_forest_word2vec_2, "F1 Score:", score_random_forest_word2vec_2)
print("Random Forest - GloVe - Accuracy:", accuracy_random_forest_glove, "F1 Score:", score_random_forest_glove)
# print("Random Forest - FastText - Accuracy:", accuracy_random_forest_fasttext, "F1 Score:", score_random_forest_fasttext)


Random Forest - Word2Vec - Accuracy: 0.7 F1 Score: 0.6875
Random Forest - Word2Vec 2 - Accuracy: 0.545 F1 Score: 0.5027322404371584
Random Forest - GloVe - Accuracy: 0.47 F1 Score: 0.48543689320388345


In [39]:
print("Logistic Regression - Word2Vec - Accuracy:", accuracy_LR_word2vec, "F1 Score:", score_LR_word2vec)
print("Logistic Regression - Word2Vec 2 - Accuracy:", accuracy_LR_word2vec_2, "F1 Score:", score_LR_word2vec_2)
print("Logistic Regression - GloVe - Accuracy:", accuracy_LR_glove, "F1 Score:", score_LR_glove)
# print("Logistic Regression - FastText - Accuracy:", accuracy_LR_fasttext, "F1 Score:", score_LR_fasttext)

Logistic Regression - Word2Vec - Accuracy: 0.735 F1 Score: 0.7195767195767195
Logistic Regression - Word2Vec 2 - Accuracy: 0.535 F1 Score: 0.5550239234449761
Logistic Regression - GloVe - Accuracy: 0.52 F1 Score: 0.5151515151515151


In [19]:
# def demo():
    # # Set your parameters
    # path = '/content/Restaurant_Reviews.tsv'
    # language = 'english'
    # words_to_be_included = ['not', 'nor', 'no']
    # words_to_be_excluded = ['opinion']
    # no_reviews = 1000
    # test_set_size = 0.2

    # Load word embedding models
    word2vec_model_path = '/content/word2vec_model.bin'
    word2vec_2_model_path = '/content/word2vec_model_window_10_dim_150_neg_10.bin'
    glove_model_path = '/content/glove_model.txt'
    fasttext_model_path = '/content/fasttext_model.bin.vectors.npy'


    word2vec_model = load_word_embedding_model("word2vec", word2vec_model_path)
    word2vec_2_model = Word2Vec.load("word2vec 2", word2vec_2_model_path)
    glove_model = load_word_embedding_model("glove", glove_model_path)
    fasttext_model = load_word_embedding_model("fasttext", fasttext_model_path)

    # Importing the dataset
    dataset = pd.read_csv(path, delimiter='\t', quoting=3)

    # Preprocessing reviews
    corpus = clean_reviews(no_reviews, language, words_to_be_excluded, words_to_be_included, dataset)

    # Creating feature vectors for Word2Vec
    X_word2vec = create_feature_vectors(corpus, word2vec_model)

    # Creating feature vectors for Word2Vec 2
    X_word2vec_2 = create_feature_vectors(corpus, word2vec_2_model)

    # Creating feature vectors for GloVe
    X_glove = create_feature_vectors(corpus, glove_model)

    # Creating feature vectors for FastText
    X_fasttext = create_feature_vectors(corpus, fasttext_model)

    y = dataset.iloc[:, 1].values

    # Splitting the dataset into the Training set and Test set
    X_train_word2vec, X_test_word2vec, y_train, y_test = train_test_split(X_word2vec, y, test_size=test_set_size)
    X_train_word2vec_2, X_test_word2vec_2, _, _ = train_test_split(X_word2vec_2, y, test_size=test_set_size)
    X_train_glove, X_test_glove, _, _ = train_test_split(X_glove, y, test_size=test_set_size)
    X_train_fasttext, X_test_fasttext, _, _ = train_test_split(X_fasttext, y, test_size=test_set_size)

    # Naive Bayes
    print("Naive Bayes - Word2Vec:")
    accuracy_bayes_word2vec, score_bayes_word2vec = naive_bayes(X_train_word2vec, y_train, X_test_word2vec, y_test)
    print("Naive Bayes - Word2Vec 2:")
    accuracy_bayes_word2vec_2, score_bayes_word2vec_2 = naive_bayes(X_train_word2vec_2, y_train, X_test_word2vec_2, y_test)
    print("Naive Bayes - GloVe:")
    accuracy_bayes_glove, score_bayes_glove = naive_bayes(X_train_glove, y_train, X_test_glove, y_test)
    print("Naive Bayes - FastText:")
    accuracy_bayes_fasttext, score_bayes_fasttext = naive_bayes(X_train_fasttext, y_train, X_test_fasttext, y_test)

    # Random Forest
    print("Random Forest - Word2Vec:")
    accuracy_random_forest_word2vec, score_random_forest_word2vec = random_forest(1000, X_train_word2vec, y_train, X_test_word2vec, y_test)
    print("Random Forest - Word2Vec 2:")
    accuracy_random_forest_word2vec_2, score_random_forest_word2vec_2 = random_forest(1000, X_train_word2vec_2, y_train, X_test_word2vec_2, y_test)  # اضافه شده
    print("Random Forest - GloVe:")
    accuracy_random_forest_glove, score_random_forest_glove = random_forest(1000, X_train_glove, y_train, X_test_glove, y_test)
    print("Random Forest - FastText:")
    accuracy_random_forest_fasttext, score_random_forest_fasttext = random_forest(1000, X_train_fasttext, y_train, X_test_fasttext, y_test)

    # Logistic Regression
    print("Logistic Regression - Word2Vec:")
    accuracy_LR_word2vec, score_LR_word2vec = logistic_regression(X_train_word2vec, y_train, X_test_word2vec, y_test)
    print("Logistic Regression - Word2Vec 2:")
    accuracy_LR_word2vec_2, score_LR_word2vec_2 = logistic_regression(X_train_word2vec_2, y_train, X_test_word2vec_2, y_test)
    print("Logistic Regression - GloVe:")
    accuracy_LR_glove, score_LR_glove = logistic_regression(X_train_glove, y_train, X_test_glove, y_test)
    print("Logistic Regression - FastText:")
    accuracy_LR_fasttext, score_LR_fasttext = logistic_regression(X_train_fasttext, y_train, X_test_fasttext, y_test)

    print("Naive Bayes - Word2Vec - Accuracy:", accuracy_bayes_word2vec, "F1 Score:", score_bayes_word2vec)
    print("Naive Bayes - Word2Vec 2 - Accuracy:", accuracy_bayes_word2vec_2, "F1 Score:", score_bayes_word2vec_2)
    print("Naive Bayes - GloVe - Accuracy:", accuracy_bayes_glove, "F1 Score:", score_bayes_glove)
    print("Naive Bayes - FastText - Accuracy:", accuracy_bayes_fasttext, "F1 Score:", score_bayes_fasttext)

    print("Random Forest - Word2Vec - Accuracy:", accuracy_random_forest_word2vec, "F1 Score:", score_random_forest_word2vec)
    print("Random Forest - Word2Vec 2 - Accuracy:", accuracy_random_forest_word2vec_2, "F1 Score:", score_random_forest_word2vec_2)
    print("Random Forest - GloVe - Accuracy:", accuracy_random_forest_glove, "F1 Score:", score_random_forest_glove)
    print("Random Forest - FastText - Accuracy:", accuracy_random_forest_fasttext, "F1 Score:", score_random_forest_fasttext)

    print("Logistic Regression - Word2Vec - Accuracy:", accuracy_LR_word2vec, "F1 Score:", score_LR_word2vec)
    print("Logistic Regression - Word2Vec 2 - Accuracy:", accuracy_LR_word2vec_2, "F1 Score:", score_LR_word2vec_2)
    print("Logistic Regression - GloVe - Accuracy:", accuracy_LR_glove, "F1 Score:", score_LR_glove)
    print("Logistic Regression - FastText - Accuracy:", accuracy_LR_fasttext, "F1 Score:", score_LR_fasttext)





#Run

In [ ]:
demo()